<a href="https://colab.research.google.com/github/MoosaAK/Projects-Deep-Learning/blob/master/Sonar_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Google Drive 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
seed = 5
numpy.random.seed(seed)


In [0]:
!pip install -U -q PyDrive

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
link = 'https://drive.google.com/open?id=1lw3yKQevqNL4AMGTwCg7DWc2Y9VRrzMc'

In [0]:
fluff, id = link.split('=')


In [0]:
get = drive.CreateFile({'id':id}) 
get.GetContentFile('sonar.csv')  
file = pd.read_csv('sonar.csv')

data = file.values
X = data[:,0:60].astype(float)
Y = data[:,60]

In [0]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

In [0]:
def create_baseline():
	model = Sequential()
	model.add(Dense(60, input_dim=60, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [0]:
# def create_baseline():
# 	model = Sequential()
# 	model.add(Dense(60, input_dim=60, activation='relu'))
#   model.add(Dense(60, input_dim=60, activation='tanh'))
#   model.add(Dense(1, activation='sigmoid'))
# 	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# 	return model

In [0]:
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, encoded_Y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/50
185/185 [==============================] - 4s 24ms/step - loss: 0.6738 - acc: 0.5676
Epoch 2/50
185/185 [==============================] - 0s 359us/step - loss: 0.6314 - acc: 0.6541
Epoch 3/50
185/185 [==============================] - 0s 364us/step - loss: 0.6002 - acc: 0.6865
Epoch 4/50
185/185 [==============================] - 0s 379us/step - loss: 0.5742 - acc: 0.7297
Epoch 5/50
185/185 [==============================] - 0s 414us/step - loss: 0.5513 - acc: 0.7622
Epoch 6/50
185/185 [==============================] - 0s 359us/step - loss: 0.5259 - acc: 0.7514
Epoch 7/50
185/185 [==============================] - 0s 377us/step - loss: 0.5101 - acc: 0.7946
Epoch 8/50
185/185 [==============================] - 0s 375us/step - loss: 0.4974 - acc: 0.7622
Epoch 9/50
185/185 [==============================] - 0s 403us/step - loss: 0.4834 - acc: 0.7892
Epoch 10/50
185/185 [==============================] - 0s 364us/step - loss: 0.4694 - acc: 0.7730
Epoch 11/50
185/185 [=========

In [0]:




estimator = KerasClassifier(build_fn=create_baseline, epochs=50, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X, encoded_Y, cv=kfold)
print("Baseline:- %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Baseline:- 83.18% (5.53%)


In [0]:
def create_smaller():
	model = Sequential()
	model.add(Dense(30, input_dim=60, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_smaller, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Smaller: 84.54% (6.05%)


In [0]:
def create_larger():
	model = Sequential()
	model.add(Dense(60, input_dim=60, activation='relu'))
	model.add(Dense(30, activation='relu'))
	model.add(Dense(1, activation='sigmoid'))
	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Larger: 87.04% (6.21%)


Larger: 88.42% (3.30%)


In [0]:
encode_Y=pandas.get_dummies(Y)
# create model, write code below
from keras import models
from keras import layers
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(60, activation='relu', input_shape=(60,)))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
    return model
k = 10
num_val_samples = len(X) // k
num_epochs = 100
all_scores = []
for i in range(k):
  print('processing fold #', i)
  val_data = X[i * num_val_samples: (i + 1) * num_val_samples]
  val_targets = encoded_Y[i * num_val_samples: (i + 1) * num_val_samples]
  partial_train_data = numpy.concatenate([X[:i * num_val_samples],X[(i + 1) * num_val_samples:]],axis=0)
  partial_train_targets = numpy.concatenate([encoded_Y[:i * num_val_samples],encoded_Y[(i + 1) * num_val_samples:]],axis=0)
  model = build_model()
  model.fit(partial_train_data, partial_train_targets,
  epochs=num_epochs, batch_size=1, verbose=0)
  val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=0)
  all_scores.append(val_mae)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3
processing fold # 4
processing fold # 5
processing fold # 6
processing fold # 7
processing fold # 8
processing fold # 9
